In [15]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit.circuit.library import ZFeatureMap, EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_ibm_provider import least_busy
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import SLSQP, L_BFGS_B
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import Options
from qiskit_ibm_runtime import QiskitRuntimeService
from sklearn.decomposition import PCA
from qiskit.algorithms.gradients import LinCombSamplerGradient

# Create sampler object
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q-asu/main/pi-deluca',
)
backend = service.backend('ibmq_qasm_simulator')
options = Options()
options.execution.shots = 512
sampler = Sampler(session=backend, options=options)
num_qubits = 6
sampling_fraction = 0.005

# Load the dataset
covtype = datasets.fetch_covtype()
X = covtype.data
y = covtype.target
classes = len(np.unique(y))

# Shuffle and select a sample from the dataset
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
sample_size = int(X.shape[0] * sampling_fraction)
X_sample, y_sample = X[indices[:sample_size]], y[indices[:sample_size]]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)

# Preprocess
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Dimensionality reduction
pca = PCA(n_components=num_qubits)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Define the feature map and ansatz
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)
ansatz = EfficientSU2(num_qubits=num_qubits, reps=1)

# Quantum circuit
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

def interpret_measurement(x):
    return x % classes

# Set up the sampler qnn
qnn = SamplerQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=interpret_measurement,
    output_shape=classes,
    sampler=sampler,
    gradient=LinCombSamplerGradient(sampler)
)

# Set up the neural network classifier
classifier = NeuralNetworkClassifier(
    qnn,
    loss='cross_entropy',
    one_hot=True,
    optimizer=L_BFGS_B(maxiter=1000),
    warm_start=True
)

# Train the classifier
classifier.fit(X_train, y_train)

# Test the classifier
accuracy = classifier.score(X_test, y_test)
print("Test accuracy:", accuracy)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Traceback (most recent call last):
  Cell In[15], line 87
    classifier.fit(X_train, y_train)
  File /opt/conda/lib/python3.10/site-packages/qiskit_machine_learning/algorithms/trainable_model.py:201 in fit
    self._fit_result = self._fit_internal(X, y)
  File /opt/conda/lib/python3.10/site-packages/qiskit_machine_learning/algorithms/classifiers/neural_network_classifier.py:111 in _fit_internal
    return self._minimize(X, y)
  File /opt/conda/lib/python3.10/site-packages/qiskit_machine_learning/algorithms/classifiers/neural_network_classifier.py:127 in _minimize
    return self._optimizer.minimize(
  File /opt/conda/lib/python3.10/site-packages/qiskit/algorithms/optimizers/scipy_optimizer.py:148 in minimize
    raw_result = minimize(
  File /opt/conda/lib/python3.10/site-packages/scipy/optimize/_minimize.py:699 in minimize
    res = _minimize_lbfgsb(fun, x0, args, jac, bounds,
  File /opt/conda/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:306 in _minimize_lbfgsb
    sf =